In [ ]:
import logging
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType, IntegerType
import psycopg2

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define schema for the flat file
flat_file_schema = StructType([
    StructField("AverageRate", FloatType(), True),
    StructField("CurrencyID", StringType(), True),
    StructField("CurrencyDate", DateType(), True),
    StructField("EndOfDayRate", FloatType(), True)
])

# Load data from the flat file source
try:
    flat_file_path = "/Volumes/genai_demo/ssis/ssis/SampleCurrencyData.txt"
    flat_file_df = spark.read.format("csv") \
        .option("header", "true") \
        .schema(flat_file_schema) \
        .load(flat_file_path)
    logger.info("Successfully loaded data from flat file.")
except Exception as e:
    logger.error(f"Error loading flat file: {e}")
    raise

# Load data from DimCurrency1 table in Unity Catalog
try:
    dim_currency_df = spark.table("sqlserver_catalog.dbo.dimcurrency1")
    logger.info("Successfully loaded data from DimCurrency1 table.")
except Exception as e:
    logger.error(f"Error loading DimCurrency1 table: {e}")
    raise

# Load data from DimDate1 table in Unity Catalog
try:
    dim_date_df = spark.table("sqlserver_catalog.dbo.dimdate1")
    logger.info("Successfully loaded data from DimDate1 table.")
except Exception as e:
    logger.error(f"Error loading DimDate1 table: {e}")
    raise

# Perform lookup transformation for Currency Key
try:
    currency_lookup_df = flat_file_df.join(
        dim_currency_df,
        flat_file_df.CurrencyID == dim_currency_df.CurrencyAlternateKey,
        "inner"
    ).select(
        flat_file_df.AverageRate,
        flat_file_df.EndOfDayRate,
        flat_file_df.CurrencyID,
        flat_file_df.CurrencyDate,
        dim_currency_df.CurrencyAlternateKey
    )
    logger.info("Successfully performed currency key lookup.")
except Exception as e:
    logger.error(f"Error in currency key lookup: {e}")
    raise

# Perform lookup transformation for Date Key
try:
    final_df = currency_lookup_df.join(
        dim_date_df,
        currency_lookup_df.CurrencyDate == dim_date_df.FullDateAlternateKey,
        "inner"
    ).select(
        currency_lookup_df.AverageRate,
        currency_lookup_df.EndOfDayRate,
        currency_lookup_df.CurrencyID,
        currency_lookup_df.CurrencyDate,
        currency_lookup_df.CurrencyAlternateKey,
        dim_date_df.FullDateAlternateKey,
        dim_date_df.DateKey
    )
    logger.info("Successfully performed date key lookup.")
except Exception as e:
    logger.error(f"Error in date key lookup: {e}")
    raise

# Write the transformed data to the target Unity Catalog table
try:
    target_table = "sqlserver_catalog.dbo.SampleOLEDBDestination"
    spark.sql(f"DROP TABLE IF EXISTS {target_table}")
    final_df.write.format("delta").mode("overwrite").saveAsTable(target_table)
    logger.info("Successfully wrote data to the target table.")
except Exception as e:
    logger.error(f"Error writing to target table: {e}")
    raise
